In [ ]:
### Uncomment the following lines to install the required packages###
# ! pip install "autogen-ext[openai]"
# ! pip install autogen-agentchat
# ! pip install azure-ai-projects 
# ! pip install azure-identity

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
import os

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='Your Project connection string from Azure Foundry here',
)

In [ ]:

with project_client:

    code_interpreter = CodeInterpreterTool()
    
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="simple-blog-writer-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        # tool_resources=code_interpreter.resources,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="""
        
        You are my Python programming assistant. Generate code and execute it according to the following requirements

        1. Save "This is blog save example" to blog-{YYMMDDHHMMSS}.md

        2. give me the download this file link
            """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # # delete the original file from the agent to free up space (note: this does not delete your version of the file)
    # project_client.agents.delete_file(file.id)
    # print("Deleted file")

    # print the messages from the agent
    messages = project_client.agents.list_messages(thread_id=thread.id)


    # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_role("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    # print(f"File: {messages.file_path_annotations}")


    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")